In [ ]:
import numpy as np
import tensorflow as tf
from cnn import CNNClassifier

tf.logging.set_verbosity(tf.logging.INFO)


In [ ]:
images_shape = [28, 28, 1]
convolutional_layers = [
            {'filters': 32,
            'kernel_size': [5, 5],
            'padding': 'same',
            'activation': 'relu'},
            {'filters': 64,
            'kernel_size': [5, 5],
            'padding': 'same',
            'activation': 'relu'},
            ]

pooling_layers = [
            {'pool_size': [2, 2],
            'strides': 2},
            {'pool_size': [2, 2],
            'strides': 2},
            ]
dense_layers = [
            {'units': 1024,
            'activation': 'relu',
            'dropout': 0.4},
            ]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
mnist_classifier = CNNClassifier(images_shape=images_shape,
                                 convolutional_layers=convolutional_layers,
                                 pooling_layers=pooling_layers,
                                 dense_layers=dense_layers,
                                 n_classes=10,
                                 optimizer=optimizer,
                                 model_dir="/tmp/mnist_convnet_model")

In [ ]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
import pandas as pd
pd.DataFrame(train_data).shape


In [ ]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)


In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=train_data,
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=200)
    #hooks=[logging_hook])


In [ ]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=eval_data,
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)